In [1]:
import pandas as pd
import re
import string
!pip install unidecode
import unidecode
from nltk import word_tokenize
from nltk.stem import SnowballStemmer
import nltk

     |████████████████████████████████| 235 kB 17.9 MB/s 


In [2]:
nltk.download('punkt')
nltk.download('stopwords')
stemmer = SnowballStemmer('spanish')
pd.set_option('max_colwidth', 1000)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [31]:
tweets = pd.read_csv('assets/tweets.csv', encoding='utf-8', sep=',')
# We just select textual content for the moment. Other columns shall be used later on
tweets = tweets[['Text', 'Candidate']].copy()
initial_size = len(tweets.index)
tweets.dropna(inplace=True)
tweets.rename({'Candidate': 'Label'}, axis='columns', inplace=True)
tweets['Label'] = tweets['Label'].map({'Petro': 0, 'Fajardo': 1, 'Uribe': 2})

In [32]:
tweets.head()

,Text,Label
0,"#Ahora realizamos recorrido de seguridad en Santa Monica, sector Toberin, localidad de #Usaquén. \n\nJunto \n@PoliciaBogota\n, \n@Uaesp\n, \n@SeguridadBOG\n, \n@SectorMovilidad\n revisamos condiciones del sector. \n\nImportantes compromisos: instalación de luminarias y recuperación espacio público.",2
1,"Hoy, en caravana de moteros junto a \n@jjUscategui\n, rendimos homenaje a los héroes de nuestra Fuerza Pública asesinados este año. \n\nQue el rugido de estas motos sirva de llamado para que cese todo acto de violencia contra nuestros militares y policías. \n\n¡No están solos!",2
2,"Acuerdo La Habana obliga al PENIS (Sustitución droga);\n\nCada familia recibe 35 mill en 5 años. Total 100 mil flias y 3,5 bill.\n\nResultado, crecen cultivos.\n\nDenunciar y reformar: condicionar a que por esfuerzo colectivo no haya droga en el entorno.",2
3,"El POT una mentira más de Claudia López. \n\nLa meta es generar cerca de 171 hectáreas de espacio público, pero ¿cómo lo hará? sí habilita más de 6.900 hectáreas para la renovación urbana. \n\nEntre más aumenta la densidad, menos espacio público habrá.\n#PilasConElPOT",2
4,#UribeConLaVerdad - Sobre el Caso de Cajamarca: “El Ministerio de Defensa va a apersonarse de los legítimos intereses de todas estas personas llenas de dolor” \n@AlvaroUribeVel\n el 12 de abril de 2004,2


In [33]:
diff = initial_size - len(tweets.index)
print(f'{diff} records were lost; that is {diff/initial_size}% of the dataset.')

4 records were lost; that is 0.0013333333333333333% of the dataset.


In [34]:
def depure(text):
    text = text.lower()
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text) # Remove links and url's
    text = re.sub('#\S+', ' ', text) # Remove hashtags
    text = re.sub('@\S+', ' ', text) # Remove Twitter mentions
    text = re.sub('\w*\d\w*', ' ', text) # Remove words with numbers
    text = unidecode.unidecode(text) # More importantly, remove accents
    text = re.sub(r'[^\w\s]', ' ', text) # Remove puntuaction
    text = re.sub('\n', ' ', text) # Remove filthy, inconvinient new lines >:)))
    return text

In [35]:
spanish_stopwords = nltk.corpus.stopwords.words('spanish')
def remove_stopwords(words):
    return [word for word in words if word not in spanish_stopwords]

In [36]:
def stem(words):
    return [stemmer.stem(word) for word in words]

In [37]:
def process(text):
    depured_text = depure(text)
    tokens = word_tokenize(depured_text)
    non_stopwords = remove_stopwords(tokens)
    stemmed = stem(non_stopwords)
    joined = ' '.join(stemmed)
    return joined

In [38]:
tweets['Text'] = tweets['Text'].apply(process)

In [39]:
tweets.head()

,Text,Label
0,realiz recorr segur sant monic sector toberin local junt revis condicion sector import compromis instalacion luminari recuperacion espaci public,2
1,hoy caravan moter junt rend homenaj her fuerz public asesin ano rug mot sirv llam ces acto violenci militar polici estan sol,2
2,acuerd haban oblig penis sustitucion drog cad famili recib mill anos total mil fli bill result crec cultiv denunci reform condicion esfuerz colect drog entorn,2
3,pot mentir mas claudi lopez met gener cerc hectar espaci public har si habilit mas hectar renovacion urban mas aument densid men espaci public habr,2
4,cas cajamarc ministeri defens va aperson legit interes tod person llen dolor abril,2


In [40]:
texts = tweets['Text']
labels = tweets['Label']

In [41]:
from keras.models import Sequential
from keras import layers
from keras.optimizers import rmsprop_v2, adam_v2
from keras.preprocessing.text import Tokenizer
from keras import regularizers
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint

from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [42]:
NUM_CLASSES = 3
MAX_WORDS = 60

In [43]:
# Let's pray on the fact that I'm not crazy and I know exactly what am I doing
NUM_CLASSES = 3
tokenizer = Tokenizer(num_words=MAX_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
processed_texts = pad_sequences(sequences)

In [44]:
X_train, X_test, y_train, y_test = train_test_split(processed_texts, labels, random_state=0)

In [45]:
y_train_oh = to_categorical(y_train, NUM_CLASSES)
y_test_oh = to_categorical(y_test, NUM_CLASSES)

In [ ]:
y_train_oh

In [48]:
model = Sequential()
model.add(layers.Embedding(MAX_WORDS, 20))
model.add(layers.LSTM(15, dropout=0.5))
model.add(layers.Dense(NUM_CLASSES, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train_oh, epochs=70, validation_data=(X_test, y_test_oh))

Epoch 1/70
71/71 [==============================] - 4s 18ms/step - loss: 1.0974 - accuracy: 0.3623 - val_loss: 1.0960 - val_accuracy: 0.3872
Epoch 2/70
71/71 [==============================] - 1s 10ms/step - loss: 1.0898 - accuracy: 0.4219 - val_loss: 1.0907 - val_accuracy: 0.3698
Epoch 3/70
71/71 [==============================] - 1s 9ms/step - loss: 1.0754 - accuracy: 0.4513 - val_loss: 1.0804 - val_accuracy: 0.4099
Epoch 4/70
71/71 [==============================] - 1s 9ms/step - loss: 1.0534 - accuracy: 0.4709 - val_loss: 1.0747 - val_accuracy: 0.4005
Epoch 5/70
71/71 [==============================] - 1s 10ms/step - loss: 1.0317 - accuracy: 0.4784 - val_loss: 1.0742 - val_accuracy: 0.4059
Epoch 6/70
71/71 [==============================] - 1s 11ms/step - loss: 1.0141 - accuracy: 0.4717 - val_loss: 1.0613 - val_accuracy: 0.4499
Epoch 7/70
71/71 [==============================] - 1s 11ms/step - loss: 1.0021 - accuracy: 0.4913 - val_loss: 1.0644 - val_accuracy: 0.4486
Epoch 8/70
71/7